In [1]:
import pandas as pd
from sklearn.utils.validation import check_X_y, check_array, check_is_fitted
import numpy as np
from joblib import dump, load
import category_encoders as ce
import pickle

In [2]:
data = pd.read_csv(r"C:\\Users\\DELL\\Downloads\\german.data",delimiter=' ', header=None)

data.columns = ['Checking Account', 'Duration', 'Credit History', 'Purpose', 'Credit Amount', 'Savings Account', 'Employment Length', 'Installment Rate', 'Personal Status and Sex', 'Other Debtors', 'Residence Since', 'Property', 'Age', 'Other Installments', 'Housing', 'Number of Credits at this bank', 'Job', 'Number of People Liable', 'Telephone', 'Foreign Worker', 'Good/Bad']


In [4]:
applicant = data[9:10]

In [5]:
## make a pandas dataframe of the input with these columns

applicant.columns = ['Checking Account', 'Duration', 'Credit History', 'Purpose', 'Credit Amount', 'Savings Account', 'Employment Length', 'Installment Rate', 'Personal Status and Sex', 'Other Debtors', 'Residence Since', 'Property', 'Age', 'Other Installments', 'Housing', 'Number of Credits at this bank', 'Job', 'Number of People Liable',  , 'Good/Bad']

applicant['Duration']= (applicant['Duration']/10.0).astype(int)
applicant['Age']= (applicant['Age']/15.0).astype(int)

applicant = applicant.drop(['Purpose', 'Employment Length', 'Installment Rate', 'Residence Since', 'Other Installments', 'Number of Credits at this bank','Job', 'Number of People Liable', 'Telephone',  ], axis=1)

encoder= ce.OrdinalEncoder(cols=['Checking Account'],return_df=True, mapping=[{'col':'Checking Account','mapping':{'A11':0,'A12':1,'A13':2,'A14':3}}])
applicant_transformed = encoder.fit_transform(applicant)

encoder= ce.OrdinalEncoder(cols=['Credit History'],return_df=True, mapping=[{'col':'Credit History','mapping':{'A30':0,'A31':1,'A32':2,'A33':3, 'A34':4}}])
applicant_transformed = encoder.fit_transform(applicant_transformed)

encoder= ce.OrdinalEncoder(cols=['Savings Account'],return_df=True, mapping=[{'col':'Savings Account','mapping':{'A61':0,'A62':1,'A63':2,'A64':3, 'A65':4}}])
applicant_transformed = encoder.fit_transform(applicant_transformed)

#Using saved one hot encoder
#------------------------------------------------------------------------------------------------------------------------------

encoder = load('personal_status_and_sex.joblib')
applicant_transformed = encoder.transform(applicant_transformed)

encoder = load('other_debtors.joblib')
applicant_transformed = encoder.transform(applicant_transformed)

#-------------------------------------------------------------------------------------------------------------------------------              
encoder= ce.OrdinalEncoder(cols=['Property'],return_df=True, mapping=[{'col':'Property','mapping':{'A121':0,'A122':1,'A123':2,'A124':3}}])
applicant_transformed = encoder.fit_transform(applicant_transformed)

encoder= ce.OrdinalEncoder(cols=['Housing'],return_df=True, mapping=[{'col':'Housing','mapping':{'A152':0,'A151':1,'A153':2}}])
applicant_transformed = encoder.fit_transform(applicant_transformed)

encoder= ce.OrdinalEncoder(cols=['Foreign Worker'],return_df=True, mapping=[{'col':'Foreign Worker','mapping':{'A201':0,'A202':1}}])
applicant_transformed = encoder.fit_transform(applicant_transformed)


#### scale before prediction
sc = load('standard_scaler.joblib')
applicant_input = sc.transform(applicant_transformed.iloc[:, :-1].values)

<ipython-input-5-4bfabff5b545>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  applicant['Duration']= (applicant['Duration']/10.0).astype(int)
<ipython-input-5-4bfabff5b545>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  applicant['Age']= (applicant['Age']/15.0).astype(int)


In [6]:
applicant_transformed

,Checking Account,Duration,Credit History,Credit Amount,Savings Account,Personal Status and Sex_A93,Personal Status and Sex_A92,Personal Status and Sex_A91,Personal Status and Sex_A94,Other Debtors_A101,Other Debtors_A103,Other Debtors_A102,Property,Age,Housing,Foreign Worker,Good/Bad
9,1,3,4,5234,0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,2,1,0,0,2


In [7]:
def pred(estimators_, X_l_, y_l_, machine_predictions_, X, M, info=False):
        """
        Performs the CLassififerCobra aggregation scheme, used in predict method.
        Parameters
        ----------
        X: array-like, [n_features]
        M: int, optional
            M refers to the number of machines the prediction must be close to to be considered during aggregation.
        info: boolean, optional
            If info is true the list of points selected in the aggregation is returned.
        Returns
        -------
        result: prediction
        """

        # dictionary mapping machine to points selected
        select = {}
        for machine in estimators_:
            # machine prediction
            model = estimators_[machine]
            label = model.predict(X)
            select[machine] = set()
            # iterating from l to n
            # replace with numpy iteration
            for count in range(0, len(X_l_)):
                if machine_predictions_[machine][count] == label:
                    select[machine].add(count)

        points = []
        # count is the indice number.
        for count in range(0, len(X_l_)):
            # row check is number of machines which picked up a particular point
            row_check = 0
            for machine in select:
                if count in select[machine]:
                    row_check += 1
            if row_check == M:
                points.append(count)

        # if no points are selected, return 0
        if len(points) == 0:
            if info:
                logger.info("No points were selected, prediction is 0")
                return (0, 0)
            logger.info("No points were selected, prediction is 0")
            return 0

        # aggregate
        classes = {}
        for label in np.unique(y_l_):
            classes[label] = 0

        for point in points:
            classes[y_l_[point]] += 1

        result = int(max(classes, key=classes.get))
        prob = classes[1]/(classes[0]+classes[1])
        if info:
            return result, points, prob
        return result, prob


    
##### estimators_

def load_estimators():
    estimators_ = {}
    machine_list = ['naive_bayes', 'tree', 'knn', 'svm', 'logreg']
    for machine in machine_list:
            try:
                if machine == 'svm':
                    estimators_['svm'] = pickle.load(open('Model_svm.sav','rb'))
                if machine == 'knn':
                    estimators_['knn'] = pickle.load(open('Model_knn.sav','rb'))
                if machine == 'tree':
                    estimators_['tree'] = pickle.load(open('Model_tree.sav','rb'))
                if machine == 'logreg':
                    estimators_['logreg'] = pickle.load(open('Model_logreg.sav','rb'))
                if machine == 'naive_bayes':
                    estimators_['naive_bayes'] = pickle.load(open('Model_naive_bayes.sav','rb'))
            except ValueError:
                continue
                
    return estimators_
    
def predict( X, estimators_, X_l_, y_l_, machine_predictions_, M=None, info=False):
        """
        Performs the ClassifierCobra aggregation scheme, calls pred.
        ClassifierCobra performs a majority vote among all points which are retained by the COBRA procedure.
        Parameters
        ----------
        X: array-like, [n_features]
        M: int, optional
            M refers to the number of machines the prediction must be close to to be considered during aggregation.
        info: boolean, optional
            If info is true the list of points selected in the aggregation is returned.
        Returns
        -------
        result: prediction
        """
        X = check_array(X)

        if M is None:
            M = len(estimators_)
        if X.ndim == 1:
            return pred(estimators_, X_l_, y_l_, machine_predictions_, X.reshape(1, -1), M=M)

        result = np.zeros(len(X))
        avg_points = 0
        index = 0
        for vector in X:
            if info:
                result[index], points, prob = pred(estimators_, X_l_, y_l_, machine_predictions_, vector.reshape(1, -1), M=M, info=info)
                avg_points += len(points)
            else:
                result[index], prob = pred(estimators_, X_l_, y_l_, machine_predictions_, vector.reshape(1, -1), M=M)
            index += 1
        
        if info:
            avg_points = avg_points / len(X_array)
            return result, avg_points, prob
        
        return result, prob

    


In [8]:
estimators_ = load_estimators()
X_l_ = load('X_l_.joblib')
y_l_ = load('y_l_.joblib')
machine_predictions_ = load('machine_predictions_.joblib')



predict(applicant_input, estimators_, X_l_, y_l_, machine_predictions_)

(array([1.]), 0.7692307692307693)